In [9]:
pip install xarray

Note: you may need to restart the kernel to use updated packages.


In [3]:
import xarray as xr

ds = xr.open_dataset("C:/Users/iarreola/E484/files/gistemp1200_GHCNv4_ERSSTv5.nc.gz")
df = ds.to_dataframe()

df.head(1000)

time_bnds  tempanomaly
lat   lon    time       nv                        
-89.0 -179.0 1880-01-15 0  1880-01-01          NaN
                        1  1880-02-01          NaN
             1880-02-15 0  1880-02-01          NaN
                        1  1880-03-01          NaN
             1880-03-15 0  1880-03-01          NaN
...                               ...          ...
             1921-06-15 1  1921-07-01          NaN
             1921-07-15 0  1921-07-01          NaN
                        1  1921-08-01          NaN
             1921-08-15 0  1921-08-01          NaN
                        1  1921-09-01          NaN

[1000 rows x 2 columns]

In [50]:
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import xarray as xr

ds = xr.open_dataset("C:/Users/iarreola/E484/files/gistemp1200_GHCNv4_ERSSTv5.nc.gz")

ds

<xarray.Dataset>
Dimensions:      (lat: 90, lon: 180, time: 1702, nv: 2)
Coordinates:
  * lat          (lat) float32 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lon          (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
  * time         (time) datetime64[ns] 1880-01-15 1880-02-15 ... 2021-10-15
Dimensions without coordinates: nv
Data variables:
    time_bnds    (time, nv) datetime64[ns] ...
    tempanomaly  (time, lat, lon) float32 ...
Attributes:
    title:        GISTEMP Surface Temperature Analysis
    institution:  NASA Goddard Institute for Space Studies
    source:       http://data.giss.nasa.gov/gistemp/
    Conventions:  CF-1.6
    history:      Created 2021-11-10 10:02:57 by SBBX_to_nc 2.0 - ILAND=1200,...

First, create a time series to a yearly frequency

In [51]:
climate = ds.resample(time='Y').mean()
anomaly = climate['tempanomaly']

A NetCDF file has three basic parts: metadata, dimensions and variables. Variables contain both metadata and data. 

Data Arrays can be inspected with the following:

`df.shape`  = shows how many dimensions are present in the data, along with a different lenght for each dimension
`ds.dims` = shows what each dimension is called
`ds.info()` = provides information on the metadata present in the file.


In [38]:
df.shape

(55144800, 2)

In [36]:
ds.dims

Frozen({'lat': 90, 'lon': 180, 'time': 1702, 'nv': 2})

In [5]:
ds.info()

xarray.Dataset {
dimensions:
	lat = 90 ;
	lon = 180 ;
	time = 1702 ;
	nv = 2 ;

variables:
	float32 lat(lat) ;
		lat:standard_name = latitude ;
		lat:long_name = Latitude ;
		lat:units = degrees_north ;
	float32 lon(lon) ;
		lon:standard_name = longitude ;
		lon:long_name = Longitude ;
		lon:units = degrees_east ;
	datetime64[ns] time(time) ;
		time:long_name = time ;
		time:bounds = time_bnds ;
	datetime64[ns] time_bnds(time, nv) ;
	float32 tempanomaly(time, lat, lon) ;
		tempanomaly:long_name = Surface temperature anomaly ;
		tempanomaly:units = K ;
		tempanomaly:cell_methods = time: mean ;

// global attributes:
	:title = GISTEMP Surface Temperature Analysis ;
	:institution = NASA Goddard Institute for Space Studies ;
	:source = http://data.giss.nasa.gov/gistemp/ ;
	:Conventions = CF-1.6 ;
	:history = Created 2021-11-10 10:02:57 by SBBX_to_nc 2.0 - ILAND=1200, IOCEAN=NCDC/ER5, Base: 1951-1980 ;
}

In [21]:
df = ds.to_dataframe()

data = pd.DataFrame(df)

print(data)

                            time_bnds  tempanomaly
lat   lon    time       nv                        
-89.0 -179.0 1880-01-15 0  1880-01-01          NaN
                        1  1880-02-01          NaN
             1880-02-15 0  1880-02-01          NaN
                        1  1880-03-01          NaN
             1880-03-15 0  1880-03-01          NaN
...                               ...          ...
 89.0  179.0 2021-08-15 1  2021-09-01         0.61
             2021-09-15 0  2021-09-01         4.24
                        1  2021-10-01         4.24
             2021-10-15 0  2021-10-01         5.60
                        1  2021-11-01         5.60

[55144800 rows x 2 columns]


The following is doing the same, just using the package `netCDF4`

In [8]:
pip install netCDF4

Note: you may need to restart the kernel to use updated packages.


In [25]:
import netCDF4 as nc 
from netCDF4 import Dataset

fn = "C:/Users/iarreola/E484/files/gistemp1200_GHCNv4_ERSSTv5.nc.gz"

ds = nc.Dataset(fn)


OSError: [Errno -51] NetCDF: Unknown file format: b'C:/Users/iarreola/E484/files/gistemp1200_GHCNv4_ERSSTv5.nc.gz'